# WSGI

[pep333](https://www.python.org/dev/peps/pep-0333/)     
WSGI: Web Server Gateway Interface 。它是Web服务器和Python Web框架/应用程序之间的标准化接口。

## WSGI的目的
WSGI旨在以更统一的方式部署使用不同框架的Python Web应用程序。可以使用与实现WSGI的任何其他Web应用程序相同的方式调用实现WSGI接口的Web应用程序。使用WSGI的Web应用程序的基本调用非常简单.

### WSGI详解
WSGI接口有两个方面：“服务器”或“网关”端，以及“应用程序”或“框架”端。服务器端调用应用程序端提供的可调用对象。提供该对象的具体细节取决于服务器或网关。假设某些服务器或网关需要应用程序的部署者编写一个简短的脚本来创建服务器或网关的实例，并为其提供应用程序对象。其他服务器和网关可以使用配置文件或其他机制来指定应从哪里导入或以其他方式获得应用程序对象。

#### app / framework
应用程序对象只是一个可调用的对象，它接受两个参数。术语“对象”不应该被误解为需要实际的对象实例：具有__call__方法的函数，方法，类或实例都可以用作应用程序对象。应用程序对象必须能够被多次调用，因为几乎所有服务器/网关（CGI除外）都会发出这样的重复请求。

In [2]:
def simple_app(environ, start_response):
    """Simplest possible application object"""
    status = '200 OK'
    response_headers = [('Content-type', 'text/plain')]
    start_response(status, response_headers)
    return ['Hello world!\n']

In [3]:
class AppClass:
    def __init__(self, environ, start_response):
        self.environ = environ
        self.start = start_response

    def __iter__(self):
        status = '200 OK'
        response_headers = [('Content-type', 'text/plain')]
        self.start(status, response_headers)
        yield "Hello world!\n"

#### The Server/Gateway 
服务器或网关为从HTTP客户端接收的每个请求调用一次可调用的应用程序，该请求针对应用程序.

In [6]:
import os, sys

def run_with_cgi(application):
    environ = dict(os.environ.items())
    environ['wsgi.input']        = sys.stdin
    environ['wsgi.errors']       = sys.stderr
    environ['wsgi.version']      = (1, 0)
    environ['wsgi.multithread']  = False
    environ['wsgi.multiprocess'] = True
    environ['wsgi.run_once']     = True

    if environ.get('HTTPS', 'off') in ('on', '1'):
        environ['wsgi.url_scheme'] = 'https'
    else:
        environ['wsgi.url_scheme'] = 'http'

    headers_set = []
    headers_sent = []

    def write(data):
        if not headers_set:
             raise AssertionError("write() before start_response()")

        elif not headers_sent:
             # Before the first output, send the stored headers
             status, response_headers = headers_sent[:] = headers_set
             sys.stdout.write('Status: %s\r\n' % status)
             for header in response_headers:
                 sys.stdout.write('%s: %s\r\n' % header)
             sys.stdout.write('\r\n')

        sys.stdout.write(data)
        sys.stdout.flush()

    def start_response(status, response_headers, exc_info=None):
        if exc_info:
            try:
                if headers_sent:
                    # Re-raise original exception if headers sent
                    raise (exc_info[0], exc_info[1], exc_info[2])
            finally:
                exc_info = None     # avoid dangling circular ref
        elif headers_set:
            raise AssertionError("Headers already set!")

        headers_set[:] = [status, response_headers]
        return write

    result = application(environ, start_response)
    try:
        for data in result:
            if data:    # don't send headers until body appears
                write(data)
        if not headers_sent:
            write('')   # send headers now if body was empty
    finally:
        if hasattr(result, 'close'):
            result.close()

#### WSGI Middleware
它的作用有：
- 处理app的错误
- 提供会话支持
- 检测你的app
- 处理登陆认证
- Gzip输出

In [ ]:
# 处理web错误
from paste.exceptions.errormiddleware import ErrorMiddleware
wrapped_app = ErrorMiddleware(simple_app, global_conf, debug=False,
                              error_email='fred@example.com',smtp_server='localhost')

# uWSGI
uWSGI项目旨在全栈开发用于构建托管服务。

应用程序服务器（用于各种编程语言和协议），代理，进程管理器和监视器都使用通用的api和通用配置样式实现。

由于其可插拔架构，它可以扩展为支持更多平台和语言。

目前，您可以使用C，C ++和Objective-C编写插件。

名称中的“WSGI”部分是对同名Python标准的致敬，因为它是该项目的第一个开发插件。

多功能性，性能，低资源使用和可靠性是项目的优势（以及遵循的唯一规则）。